Cargamos librerías

In [ ]:
import pandas as pd
import numpy as np

import sklearn as sk
from sklearn import model_selection
from sklearn import ensemble
from sklearn import metrics
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA

import imblearn as im
from imblearn import under_sampling
from imblearn import over_sampling
from collections import Counter

Función para calcular las métricas accuracy, precision, recall y f1 tomado del colab de la materia.

In [ ]:
def accuracy_precision_recall(X, y):
    X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=0.5, shuffle=True, stratify=y, random_state=42)

    clf = sk.ensemble.RandomForestClassifier(max_features= 'log2',min_samples_split = 20, min_samples_leaf = 10, n_estimators=n_estimators, max_depth=max_depth, n_jobs=-1, random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    print("accuracy:", sk.metrics.accuracy_score(y_test, y_pred))
    print("precision:", sk.metrics.precision_score(y_test, y_pred))
    print("recall:", sk.metrics.recall_score(y_test, y_pred))
    print("f1:",  sk.metrics.f1_score(y_test, y_pred))

Esta función calcula cuanto gustó/no gustó un género (si género es lo que se ingresa en el campo_group_by) a un usuario tal como lo explicaron los docentes en clase
en campo_id va 'id_lector' y en campo_label 'label'.
esta versión calcula una proporción "ponderada"
proporción = (likes-dislikes) * (likes+dislikes) / total de opiniones del usuario

In [ ]:
def crear_columna_prop_ponderado(df_train, df_test, campo_group_by, campo_id, campo_label):
    df_tmp = pd.DataFrame()
    df_tmp_global = pd.DataFrame()
    df_tmp['cant_total'] = df_train.groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp_global['cant_total_global'] = df_train.groupby([campo_id],observed=False)[campo_label].size()
    df_tmp['cant_likes'] = df_train[df_train[campo_label] == 1].groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp['cant_dislikes'] = df_train[df_train[campo_label] == 0].groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp = df_tmp.fillna(0)
    df_tmp = pd.merge(df_tmp, df_tmp_global, how='left', left_index=True, right_index=True)

    df_tmp['proporcion'] = ((df_tmp['cant_likes']-df_tmp['cant_dislikes'])*df_tmp['cant_total'])/ (df_tmp['cant_total_global'])

    df_tmp['proporcion'] = df_tmp['proporcion'].fillna(value=0)

    columnas = list(df_train.groupby(campo_group_by,observed=False).groups.keys())

    col_list = ['cant_likes','cant_dislikes','cant_total','cant_total_global']
    df_tmp.drop(columns=col_list, inplace=True)
    df_tmp = df_tmp.reset_index()
    df_pivot = df_tmp.pivot(index=campo_id, columns=campo_group_by, values='proporcion')
    df_pivot = df_pivot.reset_index()
    df_pivot = df_pivot.fillna(0)


    columna_res = campo_group_by + '_' + 'prop_pond'
    columnas_campo_group_by = []
    for i in range(len(columnas)):
    	columnas_campo_group_by.append(campo_group_by + '_' + columnas[i])

    df_train = pd.merge(df_train, df_pivot, on=campo_id, how='left')
    df_train_dummies = pd.get_dummies(df_train[campo_group_by], prefix=campo_group_by)
    df_train = pd.concat([df_train, df_train_dummies], axis=1)
    for i in range(len(columnas)):
    	df_train[columnas[i]] = df_train[[columnas[i],columnas_campo_group_by[i]]].prod(axis=1)
    df_train[columna_res] = df_train[columnas].sum(axis=1)
    df_train.drop(columns=columnas_campo_group_by, inplace=True)
    df_train.drop(columns=columnas, inplace=True)

    df_test = pd.merge(df_test, df_pivot, on=campo_id, how='left')
    df_test_dummies = pd.get_dummies(df_test[campo_group_by], prefix=campo_group_by)
    df_test = pd.concat([df_test, df_test_dummies], axis=1)
    for i in range(len(columnas)):
    	df_test[columnas[i]] = df_test[[columnas[i],columnas_campo_group_by[i]]].prod(axis=1)
    df_test[columna_res] = df_test[columnas].sum(axis=1)
    df_test.drop(columns=columnas_campo_group_by, inplace=True)
    df_test.drop(columns=columnas, inplace=True)

    return df_train, df_test

Esta es la función que calcula las columnas de frecuencias.

Va a servir para crear las siguientes columnas de frecuencias:

Cuánto le gustó/no le gustó el género del libro al lector

id_lector_generolit_1 id_lector_generolit_0

Cuánto le gustó/no le gustó el autor del libro (en minúsculas) al lector

id_lector_autor_low_1 id_lector_autor_low_0

Cuánto gustó/no gustó en general el género literiario

generolit_1 generolit_0

Cuánto gustó/no gustó en general el título (en minúsculas)

titulo_low_1 titulo_low_0

Cuánto gustó/no gustó en general el autor

autor_low_1 autor_low_0

Cuánto me gusta/no me gusta indicó el usuario

id_lector_1 id_lector_0


en lista_campos_group_by van los campos que se van a cruzar si son dos: p. ej. autor x lector es ['id_lector', 'autor_low'] (low de lowercase, minúscula)
o el campo que se va a usar para agrupar, p. ej. si interesa saber cuantos likes tiene un autor independientemente del usuario, entonces lista_campos_group_by será ['autor_low']

In [ ]:
def crear_columna_frec(df_train,df_test, lista_campos_group_by, campo_label, valor_label):
    colname = '_'.join(lista_campos_group_by)
    colname = '_'.join([colname, str(valor_label)])
    df_tmp = df_train[df_train[campo_label] == valor_label].groupby(lista_campos_group_by).size().reset_index(name=colname)

    df_train = pd.merge(df_train, df_tmp, on=lista_campos_group_by, how='left')
    df_train[colname] = df_train[colname].fillna(df_train[colname].mean())
    df_train[colname] = df_train[colname].astype(int)
    df_test = pd.merge(df_test, df_tmp, on=lista_campos_group_by, how='left')
    df_test[colname] = df_test[colname].fillna(df_train[colname].mean())
    df_test[colname] = df_test[colname].astype(int)

    return df_train, df_test

frec rel crea frecuencias relativas en lugar de absolutas

In [ ]:
def crear_columna_frec_rel(df_train,df_test, lista_campos_group_by, campo_label, valor_label):
    colname_pre = '_'.join(lista_campos_group_by)
    colname = '_'.join([colname_pre, str(valor_label)])
    colname_total = '_'.join([colname,'total'])
    colname_frel = '_'.join([colname,'frel'])

    df_tmp = df_train[df_train[campo_label] == valor_label].groupby(lista_campos_group_by, observed=False).size().reset_index(name=colname)
    df_tmp_total = df_train.groupby(lista_campos_group_by, observed=False).size().reset_index(name=colname_total)


    df_tmp_full = pd.merge(df_tmp_total, df_tmp, on=lista_campos_group_by, how='left')
    df_tmp_full[colname] = df_tmp_full[colname].fillna(value=0)
    df_tmp_full[colname_frel] = df_tmp_full[colname] / df_tmp_full[colname_total]

    col_list = lista_campos_group_by + [colname_frel]
    df_tmp_full = df_tmp_full[col_list]

    df_train = pd.merge(df_train, df_tmp_full, on=lista_campos_group_by, how='left')
    df_test = pd.merge(df_test, df_tmp_full, on=lista_campos_group_by, how='left')

    df_train[colname_frel] = df_train[colname_frel].fillna(df_train[colname_frel].mean())
    df_test[colname_frel] = df_test[colname_frel].fillna(df_test[colname_frel].mean())

    return df_train, df_test


frec rel ponderado crea frecuencias "ponderadas"
proporción = (likes-dislikes)* (likes+dislikes)/(total de opiniones del lector)
o lo es lo mismo
proporción = (likes-(total-likes)) * total/ (total de opiniones del lector)


donde total es likes+dislikes. Por ejemplo, total puede ser la suma de likes y dislikes que introdujo un lector para un determinado género literario.

In [ ]:
def crear_columna_frec_rel_ponderado(df_train,df_test, lista_campos_group_by, campo_label, valor_label):
    colname_pre = '_'.join(lista_campos_group_by)
    colname = '_'.join([colname_pre, str(valor_label)])
    colname_total_global = '_'.join([colname,'total_global'])
    colname_total = '_'.join([colname,'total'])
    colname_frel = '_'.join([colname,'frel_pond'])

    df_tmp = df_train[df_train[campo_label] == valor_label].groupby(lista_campos_group_by, observed=False).size().reset_index(name=colname)
    df_tmp_total = df_train.groupby(lista_campos_group_by, observed=False).size().reset_index(name=colname_total)
    df_tmp_total_global = df_train.groupby(lista_campos_group_by[0], observed=False).size().reset_index(name=colname_total_global)


    df_tmp_full = pd.merge(df_tmp_total, df_tmp, on=lista_campos_group_by, how='left')
    df_tmp_full = pd.merge(df_tmp_full, df_tmp_total_global, on=lista_campos_group_by[0], how='left')
    df_tmp_full[colname] = df_tmp_full[colname].fillna(value=0)
    df_tmp_full[colname_frel] = (df_tmp_full[colname]- (df_tmp_full[colname_total] - df_tmp_full[colname]))*df_tmp_full[colname_total]/df_tmp_full[colname_total_global]


    col_list = lista_campos_group_by + [colname_frel]
    df_tmp_full = df_tmp_full[col_list]

    df_train = pd.merge(df_train, df_tmp_full, on=lista_campos_group_by, how='left')
    df_test = pd.merge(df_test, df_tmp_full, on=lista_campos_group_by, how='left')

    df_train[colname_frel] = df_train[colname_frel].fillna(df_train[colname_frel].mean())
    df_test[colname_frel] = df_test[colname_frel].fillna(df_test[colname_frel].mean())

    return df_train, df_test

Las siguientes funciones crean una serie de columnas por cada categoría del campo_group_by, por ejemplo, una columna por cada género literario, y cada columna contiene la proporción = likes / (likes+dislikes) , que en el caso del género literario, sería cuánto le gusta (en proporción) cada género a un usuario determinado.

Por lo tanto, estas columnas van a contener una serie de valores repetidos, ya que van a ser los mismos para cada usuario.

In [ ]:
def crear_columnas_prop(df_train, df_test, campo_group_by, campo_id, campo_label):
    df_tmp = pd.DataFrame()
    df_tmp['cant_total'] = df_train.groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp['cant_likes'] = df_train[df_train[campo_label] == 1].groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp['cant_dislikes'] = df_train[df_train[campo_label] == 0].groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp = df_tmp.fillna(0)
    df_tmp['proporcion'] = df_tmp['cant_likes']/ (df_tmp['cant_total'])
    df_tmp['proporcion'] = df_tmp['proporcion'].fillna(value=0)

    col_list = ['cant_likes','cant_dislikes']
    df_tmp.drop(columns=col_list, inplace=True)
    df_tmp = df_tmp.reset_index()
    df_pivot = df_tmp.pivot(index=campo_id, columns=campo_group_by, values='proporcion')
    df_pivot = df_pivot.reset_index()
    df_pivot = df_pivot.fillna(0)
    df_train = pd.merge(df_train, df_pivot, on=campo_id, how='left')

    df_test = pd.merge(df_test, df_pivot, on=campo_id, how='left')

    return df_train, df_test

Lo mismo que la anterior, solo que es útil para usarla con títulos o autores en lugar de género literario, ya que toma los n títulos o autores sobre los cuales hay más opiniones (no serían los más populares, sino los más "opinados").

In [ ]:
def crear_n_columnas_prop(df_train, df_test, campo_group_by, campo_id, campo_label, n):
    df_tmp = pd.DataFrame()
    df_tmp['cant_total'] = df_train.groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp['cant_likes'] = df_train[df_train[campo_label] == 1].groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp['cant_dislikes'] = df_train[df_train[campo_label] == 0].groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp = df_tmp.fillna(0)
    df_tmp['proporcion'] = df_tmp['cant_likes']/ (df_tmp['cant_total'])
    df_tmp['proporcion'] = df_tmp['proporcion'].fillna(value=0)

    colname = '_'.join([campo_group_by, 'otros'])

    top_n = df_train.groupby(campo_group_by).size().nlargest(n).index

    df_tmp = df_tmp.reset_index()
    df_tmp['grupo'] = df_tmp[campo_group_by].apply(lambda x: x if x in top_n else colname)

    col_list = ['cant_likes','cant_dislikes']
    df_tmp.drop(columns=col_list, inplace=True)
    df_tmp = df_tmp.reset_index()

    df_tmp = df_tmp[~(df_tmp['grupo'] == colname)]


    df_pivot = df_tmp.pivot(index=campo_id, columns='grupo', values='proporcion')
    df_pivot = df_pivot.reset_index()
    df_pivot = df_pivot.fillna(0)

    df_train = pd.merge(df_train, df_pivot, on=campo_id, how='left')
    df_test = pd.merge(df_test, df_pivot, on=campo_id, how='left')

    for columna in list(top_n):
        df_train[columna] = df_train[columna].fillna(0)
        df_test[columna] = df_test[columna].fillna(0)

    return df_train, df_test

En forma similar a las funciones anteriores, solo que en lugar de agregar las columnas, agrega los cant_componentes primeros componentes de un PCA a partir de las columnas como las anteriores. En lugar de seleccionar los n autores más populares, toma a todos, hace un PCA y agrega los primeros componentes a train y test.

In [ ]:
def crear_columnas_prop_pca(df_train, df_test, campo_group_by, campo_id, campo_label, cant_componentes):
    df_tmp = pd.DataFrame()
    df_tmp['cant_total'] = df_train.groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp['cant_likes'] = df_train[df_train[campo_label] == 1].groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp['cant_dislikes'] = df_train[df_train[campo_label] == 0].groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp = df_tmp.fillna(0)
    df_tmp['proporcion'] = df_tmp['cant_likes']/ (df_tmp['cant_total'])
    df_tmp['proporcion'] = df_tmp['proporcion'].fillna(value=0)

    columnas = list(df_train.groupby(campo_group_by).groups.keys())

    col_list = ['cant_likes','cant_dislikes']
    df_tmp.drop(columns=col_list, inplace=True)
    df_tmp = df_tmp.reset_index()
    df_pivot = df_tmp.pivot(index=campo_id, columns=campo_group_by, values='proporcion')
    df_pivot = df_pivot.reset_index()
    df_pivot = df_pivot.fillna(0)
    df_train_backup = df_train.copy()
    df_test_backup = df_test.copy()
    df_train = pd.merge(df_train, df_pivot, on=campo_id, how='left')
    df_test = pd.merge(df_test, df_pivot, on=campo_id, how='left')

    df_train_filtrado = df_train[columnas].fillna(0)
    df_test_filtrado = df_test[columnas].fillna(0)
    pca = PCA(n_components = cant_componentes)
    pca.fit(df_train_filtrado)
    train_scores = pca.transform(df_train_filtrado)
    test_scores = pca.transform(df_test_filtrado)
    pca_columns = [f'{campo_group_by}_PC{i+1}' for i in range(train_scores.shape[1])]
    df_train_scores = pd.DataFrame(data=train_scores, columns=pca_columns)
    df_test_scores = pd.DataFrame(data=test_scores, columns=pca_columns)
    df_train_res = pd.concat([df_train_backup, df_train_scores], axis=1)
    df_test_res = pd.concat([df_test_backup, df_test_scores], axis=1)
    return df_train_res, df_test_res

Similar a la función anterior, solo que toma los n valores más opinados de la variable categórica para hacer el PCA. Es decir, en lugar de hacer el PCA con todos los autores, lo hace con los 500 (si n=500) sobre los cuales más se ha opinado.

In [ ]:
def crear_columnas_prop_pca_acotado(df_train, df_test, campo_group_by, campo_id, campo_label, cant_componentes, n):
    df_tmp = pd.DataFrame()
    df_tmp['cant_total'] = df_train.groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp['cant_likes'] = df_train[df_train[campo_label] == 1].groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp['cant_dislikes'] = df_train[df_train[campo_label] == 0].groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp = df_tmp.fillna(0)
    df_tmp['proporcion'] = df_tmp['cant_likes']/ (df_tmp['cant_total'])
    df_tmp['proporcion'] = df_tmp['proporcion'].fillna(value=0)

    colname = '_'.join([campo_group_by, 'otros'])

    top_n = df_train.groupby(campo_group_by).size().nlargest(n).index
    columnas = list(top_n)
    df_tmp = df_tmp.reset_index()
    df_tmp['grupo'] = df_tmp[campo_group_by].apply(lambda x: x if x in top_n else colname)

    col_list = ['cant_likes','cant_dislikes']
    df_tmp.drop(columns=col_list, inplace=True)
    df_tmp = df_tmp.reset_index()

    df_tmp = df_tmp[~(df_tmp['grupo'] == colname)]
    df_tmp = df_tmp.reset_index()
    df_pivot = df_tmp.pivot(index=campo_id, columns=campo_group_by, values='proporcion')
    df_pivot = df_pivot.reset_index()
    df_pivot = df_pivot.fillna(0)
    df_train_backup = df_train.copy()
    df_test_backup = df_test.copy()
    df_train = pd.merge(df_train, df_pivot, on=campo_id, how='left')
    df_test = pd.merge(df_test, df_pivot, on=campo_id, how='left')
    df_train_filtrado = df_train[columnas].fillna(0)
    df_test_filtrado = df_test[columnas].fillna(0)
    pca = PCA(n_components = cant_componentes)
    pca.fit(df_train_filtrado)
    train_scores = pca.transform(df_train_filtrado)
    test_scores = pca.transform(df_test_filtrado)
    pca_columns = [f'{campo_group_by}_PC{i+1}' for i in range(train_scores.shape[1])]
    df_train_scores = pd.DataFrame(data=train_scores, columns=pca_columns)
    df_test_scores = pd.DataFrame(data=test_scores, columns=pca_columns)
    df_train_res = pd.concat([df_train_backup, df_train_scores], axis=1)
    df_test_res = pd.concat([df_test_backup, df_test_scores], axis=1)
    return df_train_res, df_test_res

Las siguientes funciones son iguales a las anteriores solo que calculan una versión "ponderada" de la proporción de likes.

proporción = (likes-dislikes)*(likes+dislikes)/total de opiniones del usuario
la idea de esta ponderación es que la medida elegida arroje un valor positivo si hay más likes que dislikes y negativo si hay más dislikes que likes.

Luego lo multiplica por la suma de likes y dislikes y divide por el total de opiniones del usuario para "ponderarlo" por cuánto ha opinado el usuario sobre ese autor, por ejemplo, respecto del total de sus opiniones.

In [ ]:
def crear_columnas_prop_ponderado(df_train, df_test, campo_group_by, campo_id, campo_label):
    df_tmp = pd.DataFrame()
    df_tmp_global = pd.DataFrame()
    df_tmp['cant_total'] = df_train.groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp_global['cant_total_global'] = df_train.groupby([campo_id],observed=False)[campo_label].size()
    df_tmp['cant_likes'] = df_train[df_train[campo_label] == 1].groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp['cant_dislikes'] = df_train[df_train[campo_label] == 0].groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp = df_tmp.fillna(0)
    df_tmp = pd.merge(df_tmp, df_tmp_global, how='left', left_index=True, right_index=True)

    df_tmp['proporcion'] = ((df_tmp['cant_likes']-df_tmp['cant_dislikes'])*df_tmp['cant_total'])/ (df_tmp['cant_total_global'])

    df_tmp['proporcion'] = df_tmp['proporcion'].fillna(value=0)

    col_list = ['cant_likes','cant_dislikes','cant_total','cant_total_global']
    df_tmp.drop(columns=col_list, inplace=True)
    df_tmp = df_tmp.reset_index()
    df_pivot = df_tmp.pivot(index=campo_id, columns=campo_group_by, values='proporcion')
    df_pivot = df_pivot.reset_index()
    df_pivot = df_pivot.fillna(0)
    df_train = pd.merge(df_train, df_pivot, on=campo_id, how='left')

    df_test = pd.merge(df_test, df_pivot, on=campo_id, how='left')

    return df_train, df_test

def crear_n_columnas_prop_ponderado(df_train, df_test, campo_group_by, campo_id, campo_label, n):
    df_tmp = pd.DataFrame()
    df_tmp_global = pd.DataFrame()
    df_tmp['cant_total'] = df_train.groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp_global['cant_total_global'] = df_train.groupby([campo_id],observed=False)[campo_label].size()
    df_tmp['cant_likes'] = df_train[df_train[campo_label] == 1].groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp['cant_dislikes'] = df_train[df_train[campo_label] == 0].groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp = df_tmp.fillna(0)
    df_tmp = pd.merge(df_tmp, df_tmp_global, how='left', left_index=True, right_index=True)

    df_tmp['proporcion'] = ((df_tmp['cant_likes']-df_tmp['cant_dislikes'])*df_tmp['cant_total'])/ (df_tmp['cant_total_global'])

    df_tmp['proporcion'] = df_tmp['proporcion'].fillna(value=0)

    colname = '_'.join([campo_group_by, 'otros'])

    top_n = df_train.groupby(campo_group_by).size().nlargest(n).index

    df_tmp = df_tmp.reset_index()
    df_tmp['grupo'] = df_tmp[campo_group_by].apply(lambda x: x if x in top_n else colname)

    col_list = ['cant_likes','cant_dislikes']
    df_tmp.drop(columns=col_list, inplace=True)
    df_tmp = df_tmp.reset_index()

    df_tmp = df_tmp[~(df_tmp['grupo'] == colname)]


    df_pivot = df_tmp.pivot(index=campo_id, columns='grupo', values='proporcion')
    df_pivot = df_pivot.reset_index()
    df_pivot = df_pivot.fillna(0)

    df_train = pd.merge(df_train, df_pivot, on=campo_id, how='left')
    df_test = pd.merge(df_test, df_pivot, on=campo_id, how='left')

    for columna in list(top_n):
        df_train[columna] = df_train[columna].fillna(0)
        df_test[columna] = df_test[columna].fillna(0)

    return df_train, df_test

def crear_columnas_prop_pca_ponderado(df_train, df_test, campo_group_by, campo_id, campo_label, cant_componentes):
    df_tmp = pd.DataFrame()
    df_tmp_global = pd.DataFrame()
    df_tmp['cant_total'] = df_train.groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp_global['cant_total_global'] = df_train.groupby([campo_id],observed=False)[campo_label].size()
    df_tmp['cant_likes'] = df_train[df_train[campo_label] == 1].groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp['cant_dislikes'] = df_train[df_train[campo_label] == 0].groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp = df_tmp.fillna(0)
    df_tmp = pd.merge(df_tmp, df_tmp_global, how='left', left_index=True, right_index=True)

    df_tmp['proporcion'] = ((df_tmp['cant_likes']-df_tmp['cant_dislikes'])*df_tmp['cant_total'])/ (df_tmp['cant_total_global'])

    df_tmp['proporcion'] = df_tmp['proporcion'].fillna(value=0)

    columnas = list(df_train.groupby(campo_group_by,observed=False).groups.keys())

    col_list = ['cant_likes','cant_dislikes']
    df_tmp.drop(columns=col_list, inplace=True)
    df_tmp = df_tmp.reset_index()
    df_pivot = df_tmp.pivot(index=campo_id, columns=campo_group_by, values='proporcion')
    df_pivot = df_pivot.reset_index()
    df_pivot = df_pivot.fillna(0)
    df_train_backup = df_train.copy()
    df_test_backup = df_test.copy()
    df_train = pd.merge(df_train, df_pivot, on=campo_id, how='left')
    df_test = pd.merge(df_test, df_pivot, on=campo_id, how='left')

    df_train_filtrado = df_train[columnas].fillna(0)
    df_test_filtrado = df_test[columnas].fillna(0)
    pca = PCA(n_components = cant_componentes)
    pca.fit(df_train_filtrado)
    train_scores = pca.transform(df_train_filtrado)
    test_scores = pca.transform(df_test_filtrado)
    pca_columns = [f'{campo_group_by}_PC{i+1}' for i in range(train_scores.shape[1])]
    df_train_scores = pd.DataFrame(data=train_scores, columns=pca_columns)
    df_test_scores = pd.DataFrame(data=test_scores, columns=pca_columns)
    df_train_res = pd.concat([df_train_backup, df_train_scores], axis=1)
    df_test_res = pd.concat([df_test_backup, df_test_scores], axis=1)
    return df_train_res, df_test_res

def crear_columnas_prop_pca_acotado_ponderado(df_train, df_test, campo_group_by, campo_id, campo_label, cant_componentes, n):
    df_tmp = pd.DataFrame()
    df_tmp_global = pd.DataFrame()
    df_tmp['cant_total'] = df_train.groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp_global['cant_total_global'] = df_train.groupby([campo_id],observed=False)[campo_label].size()
    df_tmp['cant_likes'] = df_train[df_train[campo_label] == 1].groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp['cant_dislikes'] = df_train[df_train[campo_label] == 0].groupby([campo_id, campo_group_by],observed=False)[campo_label].size()
    df_tmp = df_tmp.fillna(0)
    df_tmp = pd.merge(df_tmp, df_tmp_global, how='left', left_index=True, right_index=True)

    df_tmp['proporcion'] = ((df_tmp['cant_likes']-df_tmp['cant_dislikes'])*df_tmp['cant_total'])/ (df_tmp['cant_total_global'])

    df_tmp['proporcion'] = df_tmp['proporcion'].fillna(value=0)

    colname = '_'.join([campo_group_by, 'otros'])

    top_n = df_train.groupby(campo_group_by).size().nlargest(n).index
    columnas = list(top_n)
    df_tmp = df_tmp.reset_index()
    df_tmp['grupo'] = df_tmp[campo_group_by].apply(lambda x: x if x in top_n else colname)

    col_list = ['cant_likes','cant_dislikes']
    df_tmp.drop(columns=col_list, inplace=True)
    df_tmp = df_tmp.reset_index()

    df_tmp = df_tmp[~(df_tmp['grupo'] == colname)]
    df_tmp = df_tmp.reset_index()
    df_pivot = df_tmp.pivot(index=campo_id, columns=campo_group_by, values='proporcion')
    df_pivot = df_pivot.reset_index()
    df_pivot = df_pivot.fillna(0)
    df_train_backup = df_train.copy()
    df_test_backup = df_test.copy()
    df_train = pd.merge(df_train, df_pivot, on=campo_id, how='left')
    df_test = pd.merge(df_test, df_pivot, on=campo_id, how='left')
    df_train_filtrado = df_train[columnas].fillna(0)
    df_test_filtrado = df_test[columnas].fillna(0)
    pca = PCA(n_components = cant_componentes)
    pca.fit(df_train_filtrado)
    train_scores = pca.transform(df_train_filtrado)
    test_scores = pca.transform(df_test_filtrado)
    pca_columns = [f'{campo_group_by}_PC{i+1}' for i in range(train_scores.shape[1])]
    df_train_scores = pd.DataFrame(data=train_scores, columns=pca_columns)
    df_test_scores = pd.DataFrame(data=test_scores, columns=pca_columns)
    df_train_res = pd.concat([df_train_backup, df_train_scores], axis=1)
    df_test_res = pd.concat([df_test_backup, df_test_scores], axis=1)
    return df_train_res, df_test_res


Tanto la base de datos de lectores como de libros han sido modificadas.
La de lectores tiene imputado el campo de género (sexo) con los valores más probables o razonables en función del nombre del usuario

la de libros es el resultado de un scraping de quelibroleo.com que permitió agregar el valor promedio de las valoraciones de los usuarios, y la cantidad de valoraciones para el rango de 0 a 10.

In [ ]:
DIR = "/content/drive/MyDrive/Clases/FCEN DM/datos/qll"
df_train = pd.read_csv(f"{DIR}/entrenamiento.csv")
df_test = pd.read_csv(f"{DIR}/prueba.csv")
df_lectores  = pd.read_csv(f"{DIR}/lectores.csv")
df_libros = pd.read_csv(f"{DIR}/libros-nuevo.csv")



df_train_orig = df_train.copy()
df_test_orig = df_test.copy()

Seleccionamos de train solo los lectores que aparecen en test

In [ ]:
df_train_id_lector = set(df_train['id_lector'].unique())
df_test_id_lector = set(df_test['id_lector'].unique())
id_lector_unicos_train = df_train_id_lector - df_test_id_lector
id_lector_comunes = df_train_id_lector.intersection(df_test_id_lector)
df_train = df_train[df_train['id_lector'].isin(list(id_lector_comunes))]
#df_train = df_train[~df_train['id_lector'].isin(list(id_lector_unicos_train))]

#el campo rating value está codificado como string, lo pasamos a float, e imputamos los faltantes con la media
df_libros['rating_value'] = df_libros['rating_value'].astype(float)
df_libros['rating_value'] = df_libros['rating_value'].apply(pd.to_numeric)
df_libros['rating_value'] = df_libros['rating_value'].fillna(df_libros['rating_value'].mean())

#estos campos con conteos están codificados como strings, los pasamos a entero
columnas_a_entero = ['best_rating', 'worst_rating', 'rating_count',
                    'cant_votos_1', 'cant_votos_2', 'cant_votos_3', 'cant_votos_4',
                    'cant_votos_5', 'cant_votos_6', 'cant_votos_7', 'cant_votos_8',
                    'cant_votos_9', 'cant_votos_10']
df_libros[columnas_a_entero] = df_libros[columnas_a_entero].apply(lambda x: pd.to_numeric(x, downcast='integer'))

df_libros.rename(columns={'genero': 'generolit'}, inplace=True)
df_libros.drop_duplicates(subset=['id_libro'], keep='first', inplace=True)
df_lectores['nacimiento'] = df_lectores['nacimiento'].replace(1910, np.nan)
df_lectores['nacimiento'] = df_lectores['nacimiento'].fillna(df_lectores['nacimiento'].median())
df_libros['generolit'] = df_libros['generolit'].fillna('generolit_desconocido')
df_libros['autor'] = df_libros['autor'].fillna('autor_desconocido')

df_libros['anio_edicion'] = pd.to_numeric(df_libros['anio_edicion'], errors='coerce')
df_libros['anio_edicion'] = df_libros['anio_edicion'].fillna(df_libros['anio_edicion'].median())

nbins=10
df_libros['anio_edicion_rango'] = pd.qcut(df_libros['anio_edicion'], q=nbins, labels=[str(x) for x in range(nbins)], precision=0, duplicates='drop')

Un intento de corregir algunos n/as de la base de libros, que quedó abandonado...

In [ ]:
df_libros.loc[df_libros['id_libro'] == 'el-gran-gatsby', 'titulo'] = df_libros.loc[df_libros['id_libro'] == 'el-gran-gatsby-2', 'titulo'].values[0]
df_libros.loc[df_libros['id_libro'] == 'el-gran-gatsby', 'autor'] = df_libros.loc[df_libros['id_libro'] == 'el-gran-gatsby-2', 'autor'].values[0]
df_libros.loc[df_libros['id_libro'] == 'el-gran-gatsby', 'generolit'] = df_libros.loc[df_libros['id_libro'] == 'el-gran-gatsby-2', 'generolit'].values[0]

Hacemos el join de los dataframes

In [ ]:
df_train_extendido = pd.merge(df_train, df_lectores, on='id_lector', how='left').merge(df_libros, on='id_libro', how='left')
df_test_extendido = pd.merge(df_test, df_lectores, on='id_lector', how='left').merge(df_libros, on='id_libro', how='left')

Limpiamos y amalgamamos los géneros literiarios en train

In [ ]:
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='varios', value='Varios')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Histórica Y Aventuras', value='Histórica y aventuras')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='histórica y aventuras', value='Histórica y aventuras')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Biografiás, Memorias', value='Biografías, Memorias')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Clasicos de la literatura', value='Clásicos de la literatura')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Novela Negra, Intriga, Terror', value='Novela negra')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Novela negra, intriga, terror', value='Novela negra')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='No Ficción', value='No ficción')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Ensayo', value='Estudios y ensayos')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Economía financiera', value='Economía')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Biografías', value='Biografías, Memorias')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Poesía, teatro', value='Poesía, Teatro')

#para igualar géneros en train y test
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Actores', value='Varios')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Administración y dirección empresarial', value='Empresa')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Autoayuda', value='Autoayuda Y Espiritualidad')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Ciencias Políticas Y Sociales', value='Ciencias Humanas')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Cómics', value='Cómics, Novela Gráfica')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Derecho', value='Ciencias Humanas')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Didáctica y metodología', value='Ciencias Humanas')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Dietética y nutrición', value='Varios')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Fotografía', value='Varios')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Guías De Viaje', value='Varios')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Historia moderna de España', value='Historia')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Histórica', value='Narrativa histórica')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Matemáticas divulgativas', value='Ciencias')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Medicina', value='Ciencias')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Naturaleza y ciencia', value='Ciencias')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Poesía', value='Poesía, Teatro')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Policiaca. Novela negra en bolsillo', value='Novela negra')
df_train_extendido.generolit = df_train_extendido.generolit.replace(to_replace='Televisión', value='Varios')

df_train_extendido['generolit'] = df_train_extendido['generolit'].astype('category')
df_train_extendido['genero'] = df_train_extendido['genero'].astype('category')

Limpiamos y amalgamamos los géneros literiarios en test

In [ ]:
df_test_extendido.generolit = df_test_extendido.generolit.replace(to_replace='Romántica, Erótica', value='Romántica, erótica')
df_test_extendido.generolit = df_test_extendido.generolit.replace(to_replace='varios', value='Varios')
df_test_extendido.generolit = df_test_extendido.generolit.replace(to_replace='Histórica Y Aventuras', value='Histórica y aventuras')
df_test_extendido.generolit = df_test_extendido.generolit.replace(to_replace='histórica y aventuras', value='Histórica y aventuras')
df_test_extendido.generolit = df_test_extendido.generolit.replace(to_replace='Biografiás, Memorias', value='Biografías, Memorias')
df_test_extendido.generolit = df_test_extendido.generolit.replace(to_replace='Clasicos de la literatura', value='Clásicos de la literatura')
df_test_extendido.generolit = df_test_extendido.generolit.replace(to_replace='Novela Negra, Intriga, Terror', value='Novela negra')
df_test_extendido.generolit = df_test_extendido.generolit.replace(to_replace='Novela negra, intriga, terror', value='Novela negra')
df_test_extendido.generolit = df_test_extendido.generolit.replace(to_replace='No Ficción', value='No ficción')
df_test_extendido.generolit = df_test_extendido.generolit.replace(to_replace='Ensayo', value='Estudios y ensayos')
df_test_extendido.generolit = df_test_extendido.generolit.replace(to_replace='Economía financiera', value='Economía')
df_test_extendido.generolit = df_test_extendido.generolit.replace(to_replace='Biografías', value='Biografías, Memorias')
df_test_extendido.generolit = df_test_extendido.generolit.replace(to_replace='Poesía, teatro', value='Poesía, Teatro')

#para igualar géneros en train y test
df_test_extendido.generolit = df_test_extendido.generolit.replace(to_replace='Informática', value='Ciencias')
df_test_extendido.generolit = df_test_extendido.generolit.replace(to_replace='Medicina divulgativa', value='Varios')
df_test_extendido.generolit = df_test_extendido.generolit.replace(to_replace='Política nacional', value='Varios')


Generamos dummies para género literario. Generamos una dummie para el género (sexo) del lector. Pasamos los nombres de autor y títulos de los libros a minúsculas en nuevas columnas (_low de lowercase).

In [ ]:
df_test_extendido['generolit'] = df_test_extendido['generolit'].astype('category')
df_test_extendido['genero'] = df_test_extendido['genero'].astype('category')

df_train_extendido_dummies_genero = pd.get_dummies(df_train_extendido['genero'], prefix='genero', drop_first = True)
df_test_extendido_dummies_genero = pd.get_dummies(df_test_extendido['genero'], prefix='genero', drop_first = True)
df_train_extendido_dummies_generolit = pd.get_dummies(df_train_extendido['generolit'], prefix='generolit')
df_test_extendido_dummies_generolit = pd.get_dummies(df_test_extendido['generolit'], prefix='generolit')

df_train_vars, df_test_vars = df_train_extendido, df_test_extendido

df_train_extendido['autor_low'] = df_train_extendido['autor'].str.lower()
df_test_extendido['autor_low'] = df_test_extendido['autor'].str.lower()
df_train_extendido['titulo_low'] = df_train_extendido['titulo'].str.lower()
df_test_extendido['titulo_low'] = df_test_extendido['titulo'].str.lower()
df_train_extendido['editorial_low'] = df_train_extendido['editorial'].str.lower()
df_test_extendido['editorial_low'] = df_test_extendido['editorial'].str.lower()

Creamos las columnas con frecuencias tal como describimos más arriba, usando la función crear_columna_frec.
En esta versión usamos las frecuencias absolutas: cantidad de likes y cantidad de dislikes

In [ ]:
lista_campos = [['generolit'],['titulo_low'],['autor_low']]

for campos in lista_campos:
    df_train_vars, df_test_vars = crear_columna_frec(df_train_vars, df_test_vars,campos, 'label', 1 )
    df_train_vars, df_test_vars = crear_columna_frec(df_train_vars, df_test_vars,campos, 'label', 0 )

lista_campos = [['id_lector','generolit'],['id_lector','autor_low'],['id_lector','editorial_low'],['id_lector','anio_edicion_rango']]

for campos in lista_campos:
    df_train_vars, df_test_vars = crear_columna_frec(df_train_vars, df_test_vars,campos, 'label', 1 )
    df_train_vars, df_test_vars = crear_columna_frec(df_train_vars, df_test_vars,campos, 'label', 0 )

Calculamos las columnas con proporciones de likes de género literario. Incluímos el primer eje de PCA de título, autor, editorial, año de edición.

In [ ]:
df_train_generolit, df_test_generolit = crear_columnas_prop_ponderado(df_train_vars, df_test_vars, 'generolit', 'id_lector', 'label')
df_train_titulo, df_test_titulo = crear_columnas_prop_pca_acotado_ponderado(df_train_generolit, df_test_generolit, 'titulo_low', 'id_lector', 'label',2,1500)
df_train_autor, df_test_autor = crear_columnas_prop_pca_acotado_ponderado(df_train_titulo, df_test_titulo, 'autor_low', 'id_lector', 'label',1,500)
df_train_editorial, df_test_editorial = crear_columnas_prop_pca_ponderado(df_train_autor, df_test_autor, 'editorial_low', 'id_lector', 'label',1)
df_train_anio_edicion_rango, df_test_anio_edicion_rango = crear_columnas_prop_pca_ponderado(df_train_editorial, df_test_editorial, 'anio_edicion_rango', 'id_lector', 'label',1)

Incluímos las dummies en el modelo

In [ ]:
df_train_extendido_con_dummies = pd.concat([df_train_anio_edicion_rango, df_train_extendido_dummies_genero, df_train_extendido_dummies_generolit], axis=1)
df_test_extendido_con_dummies = pd.concat([df_test_anio_edicion_rango, df_test_extendido_dummies_genero, df_test_extendido_dummies_generolit], axis=1)


df_train = df_train_extendido_con_dummies
df_test = df_test_extendido_con_dummies

df_train = df_train.select_dtypes(include=['float64', 'int64', 'int32', 'int16', 'int8', 'bool'])

Descartamos las columnas id_lector y año de edición de train. Descartamos las columnas de mejor y peor puntaje y cantidad total de opiniones

In [ ]:
col_list = ['id_lector','anio_edicion','best_rating', 'worst_rating', 'rating_count']

df_train.drop(columns=col_list, inplace=True)


## Datos a predecir
X = df_train[df_train.columns.drop('label')]
y = df_train['label']


Para ajustar los hiperparámetros, ahora comentado, no se ejecuta.

El score F1 daba muy alto, por encima de 0,9.

In [ ]:
# resultados = []

# for n_estimators in [50, 100, 500, 1000]:
#     for max_depth in [5, 10, 15, 30]:
#         print(f"{n_estimators=} -- {max_depth=}")

#         # Creamos el modelo
#         clf = sk.ensemble.RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, n_jobs=-1, random_state=42)

#         scores_train = []
#         scores_test = []

#         # Validación cruzada, 10 folds, shuffle antes, semilla aleatoria
#         kf = sk.model_selection.KFold(n_splits=10, shuffle=True, random_state=42)

#         for fold, (train_index, test_index) in enumerate(kf.split(X, y)):
#             # Partimos el fold en entrenamiento y prueba...
#             X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]

#             # Entrenamos el modelo en entramiento
#             clf.fit(X_train, y_train)

#             # Predecimos en train
#             y_pred = clf.predict(X_train)

#             # Medimos la performance de la predicción en entramiento
#             score_train = sk.metrics.f1_score(y_train, y_pred)
#             scores_train.append(score_train)

#             # Predecimos en test
#             y_pred = clf.predict(X_test)

#             # Medimos la performance de la predicción en prueba
#             score_test = sk.metrics.f1_score(y_test, y_pred)
#             scores_test.append(score_test)

#             print("\t", f"{fold=}, {score_train=} {score_test=}")
#             media_scores_entrenamiento = pd.Series(scores_train).mean()
#             std_scores_entrenamiento = pd.Series(scores_train).std()
#             media_scores_prueba = pd.Series(scores_test).mean()
#             std_scores_prueba=pd.Series(scores_test).std()

#         print(f"Media de scores en entrenamiento={media_scores_entrenamiento}, std={std_scores_entrenamiento}")
#         print(f"Media de scores en prueba={media_scores_prueba}, std={std_scores_prueba}")
#         print()
#         resultados.append([n_estimators, max_depth, media_scores_entrenamiento, std_scores_entrenamiento, media_scores_prueba, std_scores_prueba])


Realizamos un cambio en los parámetros del RandomForestClassifier tal como sugirió José Chelquer, de setear max_features = 2, a fin de reducir el overfitting.
Está comentado el código que empleamos cuando probamos los remuestreos para datos desbalanceados

In [ ]:
X_test = df_test[df_train.columns.drop('label')]

# Entrenamos el modelo usando todos los datos de entrenamiento
# TODO: Poner los valores de hiperparámetros que mejor dieron en el paso anterior
n_estimators = 1000
max_depth = 15
clf = sk.ensemble.RandomForestClassifier(max_features= 2,min_samples_split = 20, min_samples_leaf = 10, n_estimators=n_estimators, max_depth=max_depth, n_jobs=-1, random_state=42)

#enn = im.under_sampling.EditedNearestNeighbours(sampling_strategy='majority')

#X, y = enn.fit_resample(X, y)

#ros = im.over_sampling.RandomOverSampler(random_state=42, shrinkage=0.1)
#X, y = ros.fit_resample(X, y)

#clf = sk.ensemble.RandomForestClassifier(class_weight="balanced", n_estimators=n_estimators, max_depth=max_depth, n_jobs=-1, random_state=42)
clf.fit(X, y)

# Predecimos

df_test['label'] = clf.predict(X_test)
#df_test['label'] = clf.predict_proba(X_test)[:,1]

#proba1 = clf.predict_proba(X_test)[:,1]
#umbral = 0.4
#df_test['label'] = np.where(proba1 > umbral, 1, 0)


df_test['id'] = df_test['id_lector'].astype(str) + "--" + df_test['id_libro'].astype(str)
df_test.set_index('id', inplace=True)

# Creamos el dataframe para entregar
df_sol = df_test[["label"]]

# Tests de validación de la predicción antes de subirla
# Estos tests TIENEN que pasar sin error


assert df_sol.shape[0] == 10332, f"La cantidad de filas no es correcta. Es {df_sol.shape[0]} y debe ser 10332."
assert df_sol.shape[1] == 1, f"La cantidad de columnas no es correcta. Es {df_sol.shape[1]} y debe ser 1."
assert 'label' in df_sol.columns, "Falta la columna 'label'."
assert df_sol.index.name == 'id', "El índice debe llamarse 'id'."


Guardamos la respuesta, vemos la importancia de las columnas en el modelo e imprimimos la cantidad de ceros en a_predecir

In [ ]:
version = "v008i-final"
df_test['label'].to_csv(f"{DIR}/solucion-{version}.csv", index=True)

imp = pd.DataFrame({
    "feature": clf.feature_names_in_,
    "importance": clf.feature_importances_
})

pd.set_option('display.max_rows', 500)
print(imp.sort_values(by="importance", ascending=False))

print((df_test['label'] == 0).sum())
#accuracy_precision_recall(X, y)

                                  feature    importance
21                  id_lector_autor_low_0  2.024240e-01
20                  id_lector_autor_low_1  1.474982e-01
1                            rating_value  9.182590e-02
14                           titulo_low_1  6.546886e-02
15                           titulo_low_0  6.358406e-02
16                            autor_low_1  3.363039e-02
17                            autor_low_0  3.006716e-02
9                            cant_votos_8  2.695260e-02
8                            cant_votos_7  2.373243e-02
10                           cant_votos_9  2.333871e-02
7                            cant_votos_6  1.855361e-02
23              id_lector_editorial_low_0  1.784248e-02
6                            cant_votos_5  1.593923e-02
11                          cant_votos_10  1.468184e-02
60                      editorial_low_PC1  1.398134e-02
22              id_lector_editorial_low_1  1.288742e-02
61                 anio_edicion_rango_PC1  1.237